In [1]:
!git clone https://github.com/YhuoyuH/RetinexMamba.git

Cloning into 'RetinexMamba'...
remote: Enumerating objects: 3669, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 3669 (delta 65), reused 138 (delta 54), pack-reused 3514 (from 1)
Receiving objects: 100% (3669/3669), 995.33 MiB | 44.18 MiB/s, done.
Resolving deltas: 100% (72/72), done.


In [ ]:
!pip install torch==1.13.0 torchvision==0.14.0 torchaudio==0.13.0 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install packaging
!pip install timm==0.4.12
!pip install pytest chardet yacs termcolor
!pip install submitit tensorboardX
!pip install triton==2.0.0
!pip install causal_conv1d==1.0.0
!pip install mamba_ssm==1.0.1
!pip install scikit-learn matplotlib thop h5py SimpleITK scikit-image medpy
!pip install opencv-python joblib natsort tqdm tensorboard
!pip install einops gdown addict future lmdb numpy pyyaml requests scipy yapf lpips
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.4/299.4 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 27.3 MB/s eta 0:00:00
  Created wheel for basicsr: filename=basicsr-1.4.2-py3-none-any.whl size=214817 sha256=314ffec1ccb76be6dc28f65c59eb8c6c0e8e82183a416ad248868aae2e5bf034
  Stored in directory: /root/.cache/pip/wheels/9a/e3/e4/58f29bfabb622dd40b6d9839318ce5bf092062b81ca3aa19ea
Successfully built basicsr
Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
import os
import sys
import cv2
import numpy as np
import torch
import torch.nn.functional as F
from skimage import img_as_ubyte

sys.path.append(os.path.abspath("/content/RetinexMamba/"))
from basicsr.models import create_model
from basicsr.utils.options import parse
import utils

input_video = "/content/input.mp4"
output_video = "/content/output_enhanced.mp4"
weights_path = "/content/RetinexMamba/pretrained_weights/LOL_v1.pth"
yaml_path = "/content/RetinexMamba/Options/RetinexMamba_LOL_v1.yml"

opt = parse(yaml_path, is_train=False)
opt['dist'] = False
model_restoration = create_model(opt).net_g

checkpoint = torch.load(weights_path)
model_restoration.load_state_dict(checkpoint['params'])
model_restoration.eval()
model_restoration.cuda()

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

with torch.no_grad():
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        inp_img = np.float32(frame) / 255.
        inp_tensor = torch.from_numpy(inp_img).permute(2, 0, 1).unsqueeze(0).cuda()

        H, W = inp_tensor.shape[2:]
        pad_h = (4 - H % 4) % 4
        pad_w = (4 - W % 4) % 4
        inp_tensor = F.pad(inp_tensor, (0, pad_w, 0, pad_h), mode='reflect')

        restored = model_restoration(inp_tensor)
        restored = restored[:, :, :H, :W]

        restored_img = restored.squeeze(0).cpu().permute(1, 2, 0).numpy()
        restored_img = np.clip(restored_img, 0, 1)
        restored_img = img_as_ubyte(restored_img)

        out.write(restored_img)

        torch.cuda.empty_cache()

cap.release()
out.release()


Traceback (most recent call last):
  File "/content/RetinexMamba/Enhancement/test_from_dataset.py", line 11, in <module>
    from basicsr.models import create_model
  File "/usr/local/lib/python3.12/dist-packages/basicsr/__init__.py", line 4, in <module>
    from .data import *
  File "/usr/local/lib/python3.12/dist-packages/basicsr/data/__init__.py", line 22, in <module>
    _dataset_modules = [importlib.import_module(f'basicsr.data.{file_name}') for file_name in dataset_filenames]
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/basicsr/data/realesrgan_dataset.py", line 11, in <module>
    from basicsr.data.degradations import circular_lowpass_kernel, random_mixed_kernels
  File "/usr/local/lib/python3.12/d